---
author: 
  - name: Matthew Reda
    email: redam94@gmail.com
copyright: 
  holder: Matthew Reda
  year: 2024
citation: true
---

# Optimizer

> Optimizers for models

In [ ]:
#| default_exp optimizer

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pyswarms as ps
import numpy as np
import pandas as pd

In [ ]:
#| export
class Optimizer:
    
    def __init__(self, model):
        self.model = model
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()